In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### implementation of basic ae

In [41]:
class Evolutionary_Algorithm:
    __slots__ = [
        "mutation_rate",
        "crossover_rate",
        "number_of_generations",
        "population_size",
        "function_to_optimize",
        "population",
        "problem_dim",
        "hall_of_fame",
    ]

    def __init__(
        self,
        mutation_rate=0.7,
        crossover_rate=0.7,
        number_of_generations=50,
        population_size=100,
    ):
        self.population_size = population_size
        self.number_of_generations = number_of_generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.function_to_optimize = None
        self.population = None
        self.problem_dim = None
        self.hall_of_fame = []

    def _generate_population(self, n_genes):
        self.population = np.random.uniform(-10, 10, (self.population_size, n_genes))

    def _mutation(self, individual):
        for i in range(len(individual)):
            if np.random.rand() < self.mutation_rate:
                individual += np.random.normal(0, 1, self.problem_dim)
        return individual

    def _crossover(self, parent1, parent2):
        if np.random.rand() < self.crossover_rate:
            crossover_point = np.random.randint(1, self.problem_dim - 1)
            child1 = np.concatenate(
                (parent1[:crossover_point], parent2[crossover_point:])
            )
            child2 = np.concatenate(
                (parent2[:crossover_point], parent1[crossover_point:])
            )
            return child1, child2
        return parent1, parent2

    def _evaluate_population(self):
        fitness = np.zeros(len(self.population))
        for i, individual in enumerate(self.population):
            fitness[i] = self.function_to_optimize(*individual)
        self.hall_of_fame.append(
            self.population[np.argsort(fitness)[: int(self.population_size * 0.1)]]
        )
        return fitness

    def _select_individual(self, fitness):
        # roulette wheel selection
        probabilities = fitness / np.sum(fitness)
        return self.population[
            np.random.choice(range(self.population_size), p=probabilities)
        ]

    def _visualize_individual(self, individual):
        print(individual)

    def optimize(self, function_to_optimize):
        self.function_to_optimize = function_to_optimize
        self.problem_dim = function_to_optimize.__code__.co_argcount
        self._generate_population(self.problem_dim)
        eval = self._evaluate_population()
        print(self.hall_of_fame)
        for i, ev in enumerate(eval):
            print(f"Individual {self.population[i]}: {ev}")
        # for generation in range(self.number_of_generations):
        #     fitness = self._evaluate_population()
        #     new_population = np.zeros((self.population_size, self.problem_dim))
        #     for i in range(0, self.population_size, 2):
        #         parent1 = self._select_individual(fitness)
        #         parent2 = self._select_individual(fitness)
        #         child1, child2 = self._crossover(parent1, parent2)
        #         new_population[i] = self._mutation(child1)
        #         new_population[i + 1] = self._mutation(child2)
        #     self.population = new_population
        # best_individual = self.population[np.argmax(fitness)]
        # return best_individual

### definition of objective functions 

In [42]:
def rastrigin(*args):
    return 10 * len(args) + np.sum(args**2 - 10 * np.cos(2 * np.pi * args))


def rastrigin_5d(x, y, z, w, v):
    return rastrigin(x, y, z, w, v)


def quadratic_3d(x, y, z):
    return x**2 + y**2 + 2 * z**2

In [43]:
ea = Evolutionary_Algorithm()
ea.optimize(quadratic_3d)

[array([[ 0.68125069, -1.04047394,  2.31667024],
       [-1.58489023,  1.21597721, -2.30842658],
       [-3.2691351 , -1.16647464,  1.51277021],
       [ 2.43573977,  3.59852864, -1.98042901],
       [-0.79722593,  4.11832157,  2.1922604 ],
       [-1.53458775,  4.29868438,  1.80303806],
       [-0.21807676, -4.97828732, -1.64200718],
       [-0.26271064,  5.10847649,  1.97820636],
       [ 5.868157  ,  0.63331926,  1.35141275],
       [-1.72177357,  5.73318115, -1.42407445]])]
Individual [-3.09865864  0.27589999  9.77945731]: 200.95337690776594
Individual [-5.25237936  2.91693334  6.32827716]: 116.19017275347976
Individual [-7.43985117  6.82896894 -3.61845005]: 128.17256374500602
Individual [ 6.06949305 -0.59176992  2.97892612]: 54.936939198906195
Individual [-7.87300103 -0.13191424 -6.39953801]: 143.90971994764436
Individual [-6.31609095 -4.91282221  3.55101199]: 89.24819929431156
Individual [-2.61910153 -7.52054778  9.83594864]: 256.9101029427789
Individual [ 6.73909898  2.72853489 